In [1]:
!nvidia-smi

Sun Jan 21 16:41:01 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   54C    P0    73W / 300W |   1401MiB / 11264MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
_exp_name = "sample"
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [14]:
# copy and split data
import splitfolders
splitfolders.ratio("D:/data/Bacteria_data/classes/", output="D:/data/Bacteria_data/DIBaS/", ratio=(0.8, 0.1, 0.1))

Copying files: 2756 files [00:15, 178.77 files/s]


In [13]:
train_dic = {}
def dic_val(path, test=0):
    cls = 0
    for i in (os.listdir(path)):
        i = i.replace('.','')
        train_dic[i] = cls
        cls += 1 
    return train_dic

def move_floder(path):
    for i in os.listdir(path):
        for j in os.listdir(os.path.join(path,i)):
            for k in os.listdir(os.path.join(path,i,j)):
                os.rename(os.path.join(path,i,j,k),os.path.join(path,i,k))
            if j.endswith('.tif') == False:
                os.rmdir(os.path.join(path,i,j))
    return 0

def rename(path):
    for i in os.listdir(path):
        n = i.replace('.', '')
        n = n.replace('tif', '.tif')
        os.rename(path + i, path + n)
    print('file rename done!')

def crop(input_img_path, output_img_path, crop_w, crop_h):
    image = Image.open(input_img_path)
    x_max = image.size[0]
    y_max = image.size[1]
    num_width = int(x_max / crop_w)
    num_height = int(y_max / crop_h)
    for i in range(0, num_width):
        for j in range(0, num_height):
            ID = str(i)+str(j)
            BOX_LEFT, BOX_UP, BOX_RIGHT, BOX_DOWN = i*crop_w, j*crop_h, (i+1)*crop_w, (j+1)*crop_h
            box = (BOX_LEFT, BOX_UP, BOX_RIGHT, BOX_DOWN)
            crop_img = image.crop(box)
            crop_img.save(output_img_path+'_'+ID+'.tif') # .jpg??

def img_crop(path):
    for j in os.listdir(path):
        path_j = (path+j+'/')
        for i in os.listdir(path_j):
            fname = (path_j+i)
            #x = fname.split(".")[:-1]
            #output = ''
            #for i in x:
            #    output+=i
            #print('fname = ', fname)
            #print('out = ', output)
            crop(fname, fname, 1024, 766) # 512x383, 1024x766
            os.remove(fname)
    print('img_crop done!')
    #rename('D:/data/Bacteria_data/split/val/')
    #rename('D:/data/Bacteria_data/split/test/')


#dic_val('D:/data/Bacteria_data/label/')  # classes
#move_floder('D:/data/Bacteria_data/split/')     # move floder
img_crop('D:/data/Bacteria_data/classes/')  # only crop training data?

img_crop done!


In [5]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    
    # You may add some transforms here.
    # augmentation

    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

In [6]:
class BacteriaDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(BacteriaDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".tif")])
        if files != None:
            self.files = files

        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self,idx):
        fname = self.files[idx]
        #print(fname)
        im = Image.open(fname)
        im = self.transform(im)
        try:
            cls = fname.split("/")[-1].split("_")[0]
            #label = train_dic[cls]

            if cls in train_dic:
                label = train_dic[cls]
            else:
                label = -1

        except:
            print('label error')

        return im,label
        #return im,label,cls

In [7]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        # torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        # input shape [3, 128, 128] [channels, w, h]
        # torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None) linear transformation
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]

            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 33)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [78]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)

# The number of batch size.
batch_size = 128 # 8, 32, 128

# The number of training epochs.
n_epochs = 30

# If no improvement in 'patience' epochs, early stop.
patience = 30

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

In [85]:
# Construct train and valid datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = BacteriaDataset('D:/data/Bacteria_data/split/train/', tfm=train_tfm)
#for i in range(len(train_set)):
 #   print(train_set[i][2], train_set[i][1])
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)     #, pin_memory=True
valid_set = BacteriaDataset('D:/data/Bacteria_data/split/val/', tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)

In [86]:
# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

  0%|          | 0/5 [00:00<?, ?it/s]

[ Train | 001/030 ] loss = 0.13448, acc = 0.93585


  0%|          | 0/1 [00:00<?, ?it/s]

[ Valid | 001/030 ] loss = 1.11675, acc = 0.74242
[ Valid | 001/030 ] loss = 1.11675, acc = 0.74242 -> best
Best model found at epoch 0, saving model


  0%|          | 0/5 [00:00<?, ?it/s]

...

[ Valid | 029/030 ] loss = 1.29303, acc = 0.68182
[ Valid | 029/030 ] loss = 1.29303, acc = 0.68182


  0%|          | 0/5 [00:00<?, ?it/s]

[ Train | 030/030 ] loss = 0.09412, acc = 0.97188


  0%|          | 0/1 [00:00<?, ?it/s]

[ Valid | 030/030 ] loss = 1.32598, acc = 0.74242
[ Valid | 030/030 ] loss = 1.32598, acc = 0.74242


In [87]:
# Construct test datasets.
# The argument "loader" tells how torchvision reads the data.
test_label_array = []
test_set = BacteriaDataset('D:/data/Bacteria_data/split/test/', tfm=test_tfm)
for i in range(len(test_set)):
    test_label_array.append(test_set[i][1])
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

In [88]:
print(test_set[-1][1])

32


In [89]:
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
correct = 0
with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()
for i in range(len(prediction)):
    if test_label_array[i] == prediction[i]:
        correct += 1

answer = float((correct/len(prediction))*100)
print(f'predict:{prediction}')
print(f'correct:{test_label_array}')
print(f'accuracy = {answer}%')

  0%|          | 0/1 [00:00<?, ?it/s]

predict:[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9, 10, 16, 11, 11, 17, 17, 13, 13, 14, 14, 15, 19, 11, 16, 17, 17, 18, 14, 19, 19, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 9, 25, 25, 25, 27, 27, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 26, 8, 32]
correct:[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 32]
accuracy = 85.71428571428571%


In [90]:
# 4*4 (8736)
# test = 37.66233766233766%, 35.064935064935064% (batch=8, val沒分割) 31s/e = 930s
# test = 28.57142857142857%, 32.467532467532465%, 31.16883116883117%, 29.87012987012987%(batch=32, val沒分割) 24s/e = 720s
# test = 40.25974025974026%, 35.064935064935064% (batch=128, val沒分割) 22s/e = 660s

# test = 12.98701298701298%, 22.07792207792208%(batch=8, val有分割) 31s/e = 930s
# test = 31.16883116883117%, 15.58441558441558%(batch=32, val有分割) 24s/e = 720s
# test = 25.97402597402597%, 27.27272727272727%(batch=128, val有分割) 22s/e = 660s

# 2*2 (2184)
# test = 59.74025974025974%, 55.84415584415584%, 68.83116883116884%, 63.63636363636363%(batch=8, val沒分割) 10s/e = 300s
# test = 67.53246753246754%, 62.33766233766234%, 66.23376623376623%, 58.44155844155844%(batch=32, val沒分割) 8.5s/e = 255s
# test = 77.92207792207793%, 70.12987012987013%, 81.81818181818183%, 72.72727272727273%(batch=128, val沒分割) 8s/e = 240s

# test = 57.14285714285714%, 67.53246753246754%, (batch=8, val有分割) 10s/e = 300s
# test = 59.74025974025974%, 72.72727272727273%, 68.83116883116884%(batch=32, val有分割) 8.5s/e = 255s
# test = 71.42857142857143%, 66.23376623376623%, (batch=128, val有分割) 8s/e = 240s

# 1*1 (546)
# test = 72.72727272727273%, 61.038961038961034%, 76.62337662337663%, 62.33766233766234%(batch=8, val沒分割)
# test = 83.11688311688312%, 76.62337662337663%, 70.12987012987013%, 72.72727272727273%(batch=32, val沒分割)
# test = 87.01298701298701%, 83.11688311688312%, 80.51948051948052%, 85.71428571428571%(batch=128, val沒分割)